### Import the necessary database

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
#In[2]:
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprosess

In [3]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster(scale=4, cores=40, memory="200GB")

In [4]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.apply_mannkendall(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [ ]:
# Input the MMEM of SAT-OBS internal variability
dir_residuals = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_nh_Keil/'
ds_MME_HadCRUT5_1850_2022 = xr.open_mfdataset(dir_residuals + '1850_2022_Internal_subpolar_gyre_mean.nc')

In [6]:
ds_MME_HadCRUT5_1850_2022

<xarray.Dataset> Size: 3kB
Dimensions:  (year: 173)
Coordinates:
  * year     (year) int64 1kB 1850 1851 1852 1853 1854 ... 2019 2020 2021 2022
    height   float64 8B ...
    time     float64 8B ...
Data variables:
    tas      (year) float64 1kB dask.array<chunksize=(173,), meta=np.ndarray>

In [7]:
# Generate the running windows of the residuals of SAT-OBS
#       with a series of equal length with an interval of 5 years starting from 10 years to 100 years
#       and calculate the trend pattern of each segment
#       and calculate the ensemble standard deviation of the trend pattern of each interval of segments

# define the function to generate the running windows of the residuals of SAT-OBS
def generate_segments(data, segment_length):
    """
    data: 3D array with dimensions [year, lat, lon]
    segment_length: length of each segment in years
    """
    years = range(int(data['year'].min().item()), int(data['year'].max().item()) - segment_length + 2)
    print(years)
    # Initialize an empty list to store the segments
    segments = []
    
    # For each year in the range
    for year in years:
        # Extract the segment of data from that year to year + segment_length
        segment = data.sel(year=slice(str(year), str(year + segment_length - 1)))
        
        # Append this segment to the list of segments
        segments.append(segment)
    
    return segments

In [8]:
time_interval = np.arange(10, 74, 1)

In [9]:
time_interval

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73])

In [10]:
# Generate the running windows of the residuals of SAT-OBS
ICV_segments = {}
for i in time_interval:
    ICV_segments[i] = generate_segments(ds_MME_HadCRUT5_1850_2022['tas'], segment_length=i)

range(1850, 2014)
range(1850, 2013)
range(1850, 2012)
range(1850, 2011)
range(1850, 2010)
range(1850, 2009)
range(1850, 2008)
range(1850, 2007)
range(1850, 2006)
range(1850, 2005)
range(1850, 2004)
range(1850, 2003)
range(1850, 2002)
range(1850, 2001)
range(1850, 2000)
range(1850, 1999)
range(1850, 1998)
range(1850, 1997)
range(1850, 1996)
range(1850, 1995)
range(1850, 1994)
range(1850, 1993)
range(1850, 1992)
range(1850, 1991)
range(1850, 1990)
range(1850, 1989)
range(1850, 1988)
range(1850, 1987)
range(1850, 1986)
range(1850, 1985)
range(1850, 1984)
range(1850, 1983)
range(1850, 1982)
range(1850, 1981)
range(1850, 1980)
range(1850, 1979)
range(1850, 1978)
range(1850, 1977)
range(1850, 1976)
range(1850, 1975)
range(1850, 1974)
range(1850, 1973)
range(1850, 1972)
range(1850, 1971)
range(1850, 1970)
range(1850, 1969)
range(1850, 1968)
range(1850, 1967)
range(1850, 1966)
range(1850, 1965)
range(1850, 1964)
range(1850, 1963)
range(1850, 1962)
range(1850, 1961)
range(1850, 1960)
range(1850

In [11]:
type(ICV_segments)

dict

In [12]:
ICV_segments.keys()

dict_keys([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73])

In [13]:
# calculate the trend values of each segment
ICV_trend = {}
for key in ICV_segments.keys():
    print(key)
    ICV_trend[key] = [data_process.mk_test(x)[0] * 10.0 for x in ICV_segments[key]]

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [14]:
# # Assuming ICV_segments is a dictionary with segment_length as keys and list of DataArray segments as values
# max_num_segments = max(len(segments) for segments in ICV_segments.values())
# segment_lengths = ICV_segments.keys()

# # Create a new Dataset to hold the new arrays
# new_ds = xr.Dataset()

# for segment_length in segment_lengths:
#     segments_list = ICV_segments[segment_length]
#     # print(segments_list)
    
#     # Pad the segments list to have the same number of segments
#     padded_segments = segments_list.copy()
#     while len(padded_segments) < max_num_segments:
#         # Create a DataArray filled with NaNs to match the shape of the segments
#         nan_segment = xr.full_like(padded_segments[0], np.nan)
#         padded_segments.append(nan_segment)
    
#     # Create a coordinate for the new segment dimension
#     segment_coord = range(max_num_segments)
    
#     # Concatenate the padded segments with the new segment coordinate
#     concatenated = xr.concat(padded_segments, dim=segment_coord)
    
#     # Assign a specific name to the new dimension
#     concatenated = concatenated.rename({'concat_dim': 'segment'})
    
#     # Add the new DataArray to the new dataset
#     new_ds[f'ICV_segments_{segment_length}yr'] = concatenated

In [15]:
# ds_combined = xr.merge([ds_MME_HadCRUT5_1850_2022, new_ds])

In [16]:
# ds_combined

In [17]:
# check the minimum and maximum of the new variable
# ds_combined['ICV_segments_10yr'].min().values, ds_combined['ICV_segments_10yr'].max().values

In [18]:
ICV_trend

{10: [-0.2967921313699385,
  -0.26676166057304146,
  -0.22587861611992027,
  -0.28564713243190387,
  -0.28564713243190387,
  0.030059575665929983,
  -0.08232832035455724,
  0.29125874165364707,
  0.35784518361249507,
  0.35784518361249507,
  0.35784518361249507,
  0.2715117353316377,
  0.22271860288480275,
  -0.004249452607429294,
  0.017147879046899295,
  0.22271860288480275,
  -0.043637971971709835,
  0.017147879046899295,
  -0.20025154348616686,
  -0.1797895354900149,
  -0.15574806229260024,
  -0.12459595566692475,
  -0.2304881225065272,
  -0.23913705642581456,
  -0.3765685622624529,
  -0.32405732068246684,
  -0.22163820196607398,
  0.06733320181363987,
  0.06733320181363987,
  0.30886202813300134,
  0.5751468069613462,
  0.753877097679361,
  0.7185181973025079,
  0.7361359124046203,
  0.6497455768573077,
  0.2737032461320439,
  -0.18710014704074612,
  -0.3053878534077053,
  -0.4563953598140177,
  -0.364813608779325,
  -0.17254790151682337,
  -0.3328945128974332,
  -0.47359314296471

In [19]:
def check_shapes(data_dict):
    for key, value in data_dict.items():
        if isinstance(value, list):
            print(f"{key}: List of {len(value)} elements")
            for i, item in enumerate(value):
                shape = getattr(item, 'shape', 'No shape attribute')
                print(f"  Element {i}: {value}")
        else:
            shape = getattr(value, 'shape', 'No shape attribute')
            print(f"{key}: {value}")

In [20]:
# check_shapes(ICV_trend)

In [21]:
len(ICV_trend[20])

154

In [22]:
type(ICV_trend)

dict

In [ ]:
print(max(len(segments) for segments in ICV_trend.values()))

### Transform the dictionary to the dataset

In [ ]:
# Assuming ICV_segments is a dictionary with segment_length as keys and list of segments as values
max_num_segments = max(len(segments) for segments in ICV_trend.values())
segment_lengths = ICV_trend.keys()

# Create a new Dataset to hold the new arrays
new_ds = xr.Dataset()

for segment_length in segment_lengths:
    trend_list = ICV_trend[segment_length]
    # print(trend_list)
    
    # Pad the trend list to have the same number of trend
    padded_trend = trend_list.copy()
    print(type(padded_trend))
    # transform the list to DataArray
    padded_trend = [xr.DataArray(x) for i, x in enumerate(padded_trend)]
    while len(padded_trend) < max_num_segments:
        # Create a DataArray filled with NaNs to match the shape of the trend
        nan_segment = xr.full_like(padded_trend[0], np.nan)
        padded_trend.append(nan_segment)
    
    # Create a coordinate for the new segment dimension
    segment_coord = range(max_num_segments)
    
    # Concatenate the padded trend with the new segment coordinate
    concatenated = xr.concat(padded_trend, dim=segment_coord)
    
    # Assign a specific name to the new dimension
    concatenated = concatenated.rename({'concat_dim': 'segment'})
    
    # Add the new DataArray to the new dataset
    new_ds[f'ICV_trend_{segment_length}yr'] = concatenated

In [ ]:
new_ds

In [ ]:
# save the trend pattern of each segment
dir_out = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_nh_Keil/'
new_ds.to_netcdf(dir_out + 'Internal_subpolar_gyre_wrt_NH_mean_trend_segmented.nc')